In [19]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from keras import utils
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import docx
import pickle

In [20]:
# Максимальное количество слов 
num_words = 10000
# Максимальная длина абзаца
max_abzac_len = 90
# Количество классов абзацев, должно совпадать с dense см.ниже
nb_classes = 39

In [21]:
train = pd.read_csv('train.csv', encoding='latin-1', sep='|',
                    header=None, 
                    names=['class', 'text', 'udali1', 'udali2'])

train = train.drop(['udali1', 'udali2'], axis=1)  # ДИКИЙ КОСТЫЛЬНЫЙ FAILSAFE! ОТФОРМАТИТЬ ФАЙЛ ДАТАСЕТА ДО ЗАГРУЗКИ!


C:\Users\Denis\AppData\Local\Temp/ipykernel_12036/2986408390.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('output69.csv', encoding='latin-1', sep='|',


In [22]:
# Токенезируем текст
aboba = train['text']
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(aboba)
sequences = tokenizer.texts_to_sequences(aboba)


In [23]:
print(tokenizer.word_index)

{'â': 1, 'è': 2, 'ñóáñèäèè': 3, 'íà': 4, 'ðîññèéñêîé': 5, 'ôåäåðàöèè': 6, 'ïî': 7, 'ñ': 8, 'î': 9, 'ïðåäîñòàâëåíèÿ': 10, 'íàñòîÿùèõ': 11, 'ïðàâèë': 12, 'íå': 13, 'äëÿ': 14, 'èëè': 15, 'ñîîòâåòñòâèè': 16, 'çà': 17, 'îòáîðà': 18, 'ïðåäîñòàâëåíèè': 19, 'ôåäåðàëüíîãî': 20, 'ñîãëàøåíèÿ': 21, 'îðãàíèçàöèè': 22, 'ìèíèñòåðñòâî': 23, 'ìèíèñòåðñòâîì': 24, 'ðåçóëüòàòà': 25, 'äíÿ': 26, 'áþäæåòà': 27, 'çàÿâîê': 28, 'ñóáñèäèé': 29, 'ãðàíòà': 30, 'ñëó÷àå': 31, 'à': 32, 'ïðè': 33, 'ê': 34, 'ôèíàíñîâîãî': 35, 'äíåé': 36, 'äîêóìåíòîâ': 37, 'èç': 38, 'ñðåäñòâ': 39, 'òàêæå': 40, 'òîì': 41, '÷èñëå': 42, 'ñî': 43, '1': 44, 'îá': 45, 'ðàçâèòèÿ': 46, 'óñëîâèé': 47, 'òå÷åíèå': 48, 'äîñòèæåíèÿ': 49, 'çàÿâêè': 50, 'ïîðÿäêå': 51, 'ïðîìûøëåííîñòè': 52, 'ðåàëèçàöèè': 53, 'öåëè': 54, 'ðàçìåð': 55, 'ïîçäíåå': 56, 'ïîðÿäêà': 57, 'îðãàíèçàöèÿ': 58, 'ãîñóäàðñòâåííîãî': 59, 'òðåáîâàíèÿì': 60, 'ðàáî÷èõ': 61, 'ôîðìå': 62, 'îñíîâàíèè': 63, 'äî': 64, 'êîíòðîëÿ': 65, 'ñîãëàøåíèå': 66, 'áþäæåòíûõ': 67, 'äàëåå': 68, 'ñóáñèäèÿ':

In [24]:
# Подготовка тренировочного датасета
y_train = np_utils.to_categorical(train['class'] - 1, nb_classes)
x_train = pad_sequences(sequences, maxlen=max_abzac_len)

#################################################################

p.s. если кто-то это читает, то я забыл стереть сноски. не обращайте на них внимания, это чтобы не забыть штуки

In [25]:
model_cnn = Sequential()
model_cnn.add(Embedding(num_words, 32, input_length=max_abzac_len))
model_cnn.add(Conv1D(250, 5, padding='valid', activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(39, activation='softmax'))

In [26]:
model_cnn.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model_cnn.summary()    # Параметры. Удалить перед заливом в репозиторий

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 90, 32)            320000    
                                                                 
 conv1d_2 (Conv1D)           (None, 86, 250)           40250     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 250)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 256)               64256     
                                                                 
 dense_5 (Dense)             (None, 39)                10023     
                                                                 
Total params: 434,529
Trainable params: 434,529
Non-trainable params: 0
________________________________________________

In [27]:
model_cnn_save_path = 'best_model_cnn.h5'                         # Файл весов
checkpoint_callback_lstm = ModelCheckpoint(model_cnn_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [29]:
history_lstm = model_cnn.fit(x_train, 
                              y_train, 
                              epochs=5,                     # Подёргать туда-сюда
                              batch_size=1000,               # Подёргать туда-сюда
                              validation_split=0.1,         # Найди золотую середину
                              callbacks=[checkpoint_callback_lstm])

Epoch 1/5
487/487 [==============================] - ETA: 0s - loss: 0.1087 - accuracy: 0.9675
Epoch 1: val_accuracy improved from 0.00047 to 0.51774, saving model to best_model_cnn.h5
487/487 [==============================] - 111s 228ms/step - loss: 0.1087 - accuracy: 0.9675 - val_loss: 11.8167 - val_accuracy: 0.5177
Epoch 2/5
487/487 [==============================] - ETA: 0s - loss: 0.0580 - accuracy: 0.9757
Epoch 2: val_accuracy did not improve from 0.51774
487/487 [==============================] - 105s 215ms/step - loss: 0.0580 - accuracy: 0.9757 - val_loss: 12.0419 - val_accuracy: 0.5027
Epoch 3/5
487/487 [==============================] - ETA: 0s - loss: 0.0545 - accuracy: 0.9761
Epoch 3: val_accuracy did not improve from 0.51774
487/487 [==============================] - 109s 224ms/step - loss: 0.0545 - accuracy: 0.9761 - val_loss: 11.8177 - val_accuracy: 0.5038
Epoch 4/5
487/487 [==============================] - ETA: 0s - loss: 0.0527 - accuracy: 0.9765
Epoch 4: val_accurac

###################################################################################

In [47]:
# Подогнанный под формат файл для анализа. Разбить на CHTO и проче чтоб скормить нейронке
chto = pd.read_csv('output.csv', encoding='latin-1', sep='|',
                    header=None, 
                    names=['text'])

chto = pd.DataFrame(chto["text"], columns=chto.columns)

chto_sequences = tokenizer.texts_to_sequences(chto['text'])  

x_chto = pad_sequences(chto_sequences, maxlen=max_abzac_len)

In [48]:
# Анализируем абзацы
model_cnn.load_weights('best_model_cnn.h5')
prediction = model_cnn.predict(x_chto)


In [49]:
# Вывод предсказания. Не записывать в файл сразу, перерпроверь
for i in prediction:
    print(np.argmax(i))

34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
21
34
34
34
11
11
34
11
34
34
34
21
34
34
34
34
34
26
34
11
25
34
34
34
34
34
34
34
34
34
25
34
34
2
11
34
34
34
34
34
34
11
34
34
34
34
34
34
34
21
34
34
34
34
34
11
34
34
34
34
34
9
11
0
34
34
34
34
34
21
11
21
34
21
34
34
34
34
34
34
34
34
34
34
34
34
34
34
11
34
21
25
21
34
21
21
21
21
21
21
21
25
34
34
34
34
34
34
34
34
34
34
34
34
21
11
34
34
34
34
34
34
34
11
34
34
34
34
34
11
34
34
34
34
34
34
34
29
34
34
34
34
34
34
34
34
34
34
11
11
21
21
21
21
11
11
34
34
34
34
12
12
12
12
12
25
34
34
34
34
21
34
34
34
34
34
34
34
21
34
34
34
34
11
34
34
34
25
34
34
34
34
34
34
34
34
34
34
34
34
21
34
34
34
21
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
11
34
34
34
34
11
34
34
11
11
11
11
11
11
34
34
34
11
11
34
34
34
34
34
34
34
21
34
34
34
34
34
34
11
26
11
34
34
34
11
11
25
11
34
25
34
34
23
23
11
1